In [ ]:
from fastai.text.all import *
from fastai.vision.widgets import *

In [ ]:
import urllib.request

MODEL_URL = "https://github.com/mikegarts/remarqueapp/raw/master/remarque.pkl"

# Remarquefy - What would Remarque say
## A neural network trained to complete a few words of an input as a paragraph from a book by Erich Maria Remarque

In [ ]:
MIN_WORDS = 80
path = Path()
btn_generate = widgets.Button(description='Generate paragraph')
inp_text = widgets.Text(placeholder='I was drinking because')
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=MIN_WORDS * 2,
    description='Generating: ',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)

result = widgets.Textarea(rows=16)

def get_learn():
    #     return load_learner('remarque.pkl', cpu=True)
    model_filename = 'dw_' + Path(MODEL_URL).name
    if not Path(model_filename).is_file():
        result.value = 'Downloading model...'
        urllib.request.urlretrieve(MODEL_URL, model_filename)
        result.value = 'Loading model...'
    
    result.value = ''
    return load_learner(model_filename)



learn = get_learn()

def generate_until(text, min_words=MIN_WORDS, stop='.', temperature=0.75):
    added_words = 0
    
    while True:
        res = learn.predict(text, 1, temperature=temperature)    
        words_count = len(res.split())
        if (res[-1] != stop or added_words <= min_words) and (added_words < min_words * 2):
            added_words += 1
            text = res
            result.value = text
            progress.value += 1
            continue
        else:
            print('last', res[-1])
            print('added', added_words)
            print('min', min_words)
            break
    
    if res[-1] != stop:
        res += stop

    res = res.replace('xxunk','')
    res = res.replace('Xxunk','')
    res = res.replace(' i ',' I ')
    res = res.replace(' .','.')
    res = res.replace(' ,',',')
    print('added', added_words)
    return res

def on_click_generate(change):
    progress.value = 1
    btn_generate.disabled = True
    inp_text.disabled = True
    result.value = 'Generating ... please wait'
    preds = generate_until(inp_text.value)
    result.value = preds
    btn_generate.disabled = False
    inp_text.disabled = False
    progress.value = 0
    inp_text.value = ''

btn_generate.on_click(on_click_generate)
display(VBox([widgets.Label('Enter a few words to start the paragraph'),inp_text, btn_generate, progress, result]))
